<a href="https://colab.research.google.com/github/danosaku/classificacao_artigos_webofscience_mineracao_texto/blob/main/Pre_processamento_TCC_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_pretrained_bert
!pip install transformers

from pytorch_pretrained_bert import BertTokenizer, BertConfig
from keras.utils import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## preprocessing before train bert model.

from transformers import DistilBertTokenizerFast
from torch.optim import AdamW


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00


In [ ]:
import sys
import os
import nltk
import glob
from nltk.stem.porter import *
from nltk.stem import RSLPStemmer, PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
!apt install build-essential libpoppler-cpp-dev pkg-config python3-dev poppler-utils
import pandas as pd
import numpy as np

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu4).
python3-dev is already the newest version (3.8.2-0ubuntu2).
python3-dev set to manually installed.
build-essential is already the newest version (12.8ubuntu1.1).
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5 poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 220 kB of archives.
After this operation, 977 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler-cpp0v5 amd64 0.86.1-0ubuntu1.1 [35.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler-cpp-dev amd64 0.86.1-0ubuntu1.1 [10.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fe

In [ ]:
nltk.download('punkt') # punctuation
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:

def BERT_preprocessing(X, tokenizer):

    sentences = [f"[CLS] {i} [SEP]" for i in X]

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    return tokenized_texts

Download e descompactação da planilha

In [ ]:
#link do arquivo no Google Drive
#https://drive.google.com/file/d/1kFSj_Lb2XxHT1A_YO_YDRaAFtjEKTps3/view?usp=share_link


!gdown 1xDNirOca4fKFQEp1X3kDwif2lx7P6Snk
!unzip Dados_webofscience.zip



Downloading...
From: https://drive.google.com/uc?id=1xDNirOca4fKFQEp1X3kDwif2lx7P6Snk
To: /content/Dados_webofscience.zip
100% 1.36M/1.36M [00:00<00:00, 42.9MB/s]
Archive:  Dados_webofscience.zip
  inflating: Busca final 2_expressao simplificada.xls  


Abrir planilha como um dataframe


In [ ]:
dados = pd.read_excel ('Busca final 2_expressao simplificada.xls')
display(dados)
!rm Busca\ final\ 2_expressao\ simplificada.xls


,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,UT (Unique WOS ID),Web of Science Record,CLASS,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79
0,J,"Bertol, I; Gobbi, E; Barbosa, FT; Paz-Ferreiro...",NaN,NaN,NaN,"Bertol, Ildegardis; Gobbi, Ederson; Barbosa, F...",NaN,NaN,WATER EROSION IN NATURAL GRASSLAND UNDER DIFFE...,REVISTA BRASILEIRA DE CIENCIA DO SOLO,...,WOS:000295854500036,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,OBS: 0 = não se aplica; 1 = se aplica
1,J,"Xu, S; Silveira, ML; Sollenberger, LE; Viegas,...",NaN,NaN,NaN,"Xu, S.; Silveira, M. L.; Sollenberger, L. E.; ...",NaN,NaN,Conversion of native rangelands into cultivate...,JOURNAL OF SOIL AND WATER CONSERVATION,...,WOS:000427469900010,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J,"Lanzanova, ME; Eltz, FLF; Nicoloso, RD; Cassol...",NaN,NaN,NaN,"Lanzanova, Mastrangello Enivar; Foletto Eltz, ...",NaN,NaN,RESIDUAL EFFECT OF SOIL TILLAGE ON WATER EROSI...,REVISTA BRASILEIRA DE CIENCIA DO SOLO,...,WOS:000331652000025,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,J,"Freitas, MASR; Andrade, EM; Weber, OB; Palacio...",NaN,NaN,NaN,"Freitas, M. A. S. R.; Andrade, E. M.; Weber, O...",NaN,NaN,Bedload sediment and nutrient losses in agro-e...,NUTRIENT CYCLING IN AGROECOSYSTEMS,...,WOS:000325848600005,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J,"da Costa, CFG; Figueiredo, RD; Oliveira, FD; S...",NaN,NaN,NaN,"da Costa, Cristiane F. G.; Figueiredo, Ricardo...",NaN,NaN,Runoff in Oxisol under different agroecosystem...,REVISTA BRASILEIRA DE ENGENHARIA AGRICOLA E AM...,...,WOS:000328262800007,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,J,"Moldes, CA; de Lima, OF; Merini, LJ; Tsai, SM;...",NaN,NaN,NaN,"Moldes, Carlos A.; Fontao de Lima Filho, Oscar...",NaN,NaN,Occurrence of powdery mildew disease in wheat ...,ACTA PHYSIOLOGIAE PLANTARUM,...,WOS:000382675700006,View Full Record in Web of Science,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
858,J,"Silva, RD; Barioni, LG; Pellegrino, GQ; Moran, D",NaN,NaN,NaN,"Silva, Rafael De Oliveira; Barioni, Luis Gusta...",NaN,NaN,The role of agricultural intensification in Br...,AGRICULTURAL SYSTEMS,...,WOS:000425567400009,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
859,J,"Zeid, M; Yu, JK; Goldowitz, I; Denton, ME; Cos...",NaN,NaN,NaN,"Zeid, M.; Yu, J. K.; Goldowitz, I.; Denton, M....",NaN,NaN,Cross-amplification of EST-derived markers amo...,FIELD CROPS RESEARCH,...,WOS:000279096000004,View Full Record in Web of Science,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,J,"Jesus, ED; Liang, C; Quensen, JF; Susilawati, ...",NaN,NaN,NaN,"Jesus, Ederson da C.; Liang, Chao; Quensen, Jo...",NaN,NaN,"Influence of corn, switchgrass, and prairie cr...",GLOBAL CHANGE BIOLOGY BIOENERGY,...,WOS:000370492100019,View Full Record in Web of Science,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dados.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [ ]:
dados = dados.dropna(subset=['Abstract', 'CLASS'])  ## Remove linhas que não contém abstract
text = dados[['Abstract', 'Article Title', 'Author Keywords']].apply(lambda x: ' '.join(x.astype(str)), axis=1).tolist()  ## extrai o abstract, article title e authors keywors do dataframe e junta em uma única string.
text = [i.lower() for i in text]
ID = dados['UT (Unique WOS ID)'].values.tolist()
#authors = dados['Authors'].values.tolist()
#authors = [i.lower() for i in authors]
target = dados['CLASS'].values.tolist()
text = [i.lower() for i in text]
#text1 = []
#text2 = []
#for i in text:
#  total = len(i)/2
#  t1 = ""
#  t2 = ""
#  sentences = i.split('.')
#  for j in sentences:
#    if len(t1) < total:
#      t1 = t1+ j + "."
#    else:
#      t2 = t2+ j +"."
#  text1.append(t1)
#  text2.append(t2)
#text = text2


### Gera uma lista de sentenças e de classe alvo - Não vai ser usado por enquanto ( Usando em redes heterogêneas)

In [ ]:
sentences = []
target_sentences = []
for i,j in zip(text, target):
  sub_sentences = i.split('.')
  sentences = sentences + sub_sentences
  for k in range(len(sub_sentences)):
    target_sentences.append(j)



### Pré-processamento de texto (remoção de stopword e stemming)

In [ ]:
#Cria a função remove_stopwords para tokenizar o texto e remover stopwords e outras palavras desnecessárias

def remove_stopwords(text,stop_words):

  ##cria lista de palavras desnecessárias (se identificar alguma no decorrer do trabalho)

  unnecessary_words = ["c","annals","botany","company","elsevier","science","b","v","ltd","published","all",
                       "rights","reserved","sas","gmbh","authors","social","chemical","industry","saab","friends",
                       "keai","communications","co","copyright","john","wiley","sons","sal"]
  unnecessary_words = []
  # Adiciona palavras desnecessárias às stopwords

  stop_words = stop_words + unnecessary_words

  # tudo para caixa baixa

  s = str(text).lower()

  #tokeniza o texto

  tokens = word_tokenize(s)

  # remove stopwords, dígitos, caracteres especiais e pontuações
  v = [word for word in tokens if not word in stop_words and word.isalnum() and not word.isdigit()]

  return v

In [ ]:
#Cria a função stemming para recuperar a raiz das palavras

def stemming(tokens,stemmer):
  remove_list = ['differ', 'greater', 'studi', 'aim', 'quantifi', 'total', 'ii', 'treatment', 'mm', 'h', 'plot', 'without', 'well', 'howev', 'challeng', 'particularli', 'evalu',
                 'respons', 'consist', 'replic', 'repres', 'spp', 'occur', 'cm', 'g', 'kg', 'also', 'show', 'effect',
                 'taken', 'ha', 'mg', 'found', 'embrapa', 'day', 'may', 'better', 'within', 'inform', 'still', 'could', 'best', 'includ', 'wherea', 'de', 'demonstr', 'approach',
                 'across', 'main', 'number', 'conclud', 'understand', 'known', 'earli', 'overal', 'cv', 'sp', 'due', 'possibl', 'involv', 'among', 'purpos',
                 'x', 'input', 'amount', 'data', 'obtain', 'futur', 'unit', 'must', 'regard', 'allow', 'term', 'r', 'probabl', 'record', 'part',
                 'increase', 'result', 'higher', 'two', 'three', 'high', 'respect', 'observ', 'experi', 'four', 'affect', 'influenc', 'one', 'per', 'contribut', 'provid', 'work',
                 'similar', 'consid', 'five', 'suggest', 'success', 'method', 'limit', 'six', 'identifi', 'research', 'thu', 'therefor', 'altern', 'need', 'order', 'ratio', 'object word'
                 'yr', 'although', 'current', 'remain', 'would', 'except', 'matter som', 'eight', 'discuss', 'seven', 'along', 'e', 'introduc']
                 #,'around', 'becom', 'despit', 'experiment', 'explain', 'first', 'help', 'implement', 'introduct', 'knowledg', 'like', 'month', 'much', 'municip', 'open', 'option', 'paper',
                 #'promis', 'report', 'requir', 'review', 'shift', 'subject', 'vari', 'variabl', 'variou', 'year', 'yr']## remoção para testes

  remove_list = []

  tokens_stems = [stemmer.stem(word) for word in tokens]

  for l in range(len(remove_list)):
     while(remove_list[l] in tokens_stems):
       tokens_stems.remove(remove_list[l])

  ### Remove palavras consecutivas iguais
  m = 0
  while(m < (len(tokens_stems)-1)):
     if (tokens_stems[m]==tokens_stems[m+1]):
         tokens_stems.pop(m+1)
     m+=1

  return tokens_stems

In [ ]:
#cria função meu_tokenizador

def meu_tokenizador(doc, stop_words=nltk.corpus.stopwords.words('english'), stemmer=PorterStemmer()):
  tokens = remove_stopwords(doc,stop_words)
  return stemming(tokens,stemmer)


In [ ]:
def get_cluster_descriptors(VSM, df_documentos, cluster_id, max_terms=3):
  df_descritors = pd.DataFrame()
  df_descritors['word'] = VSM.get_feature_names_out()
  df_descritors['tfidf_sum'] = VSM.transform(df_documentos[df_documentos.cluster==cluster_id]['text']).toarray().sum(axis=0)
  df_descritors.sort_values(by='tfidf_sum',ascending=False,inplace=True)

  num_docs = len(df_documentos[df_documentos.cluster==cluster_id]['text'])
  descriptors =  df_descritors[df_descritors.tfidf_sum > 0].head(max_terms).word.to_list()

  return num_docs,descriptors

As funções remove_stopwords e stemming estão sendo chamadas pela função meu_tokenizador. A função meu_tokenizador é utilizada na função para gerar os embbedings dos textos como no código abaixo. Você pode usar o CountVectorizer ou o TfidfVectorizer. O VSM.fit(texts) faz o treinamento do modelo para gerar os embbeddings. VSM.vocabulary_ mostra os n-gramas encontrados.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#ngram = (1,3) ## n-gramas de tamanho 1 (unigrama) até 3 (tri-grama)
#VSM = TfidfVectorizer(tokenizer=meu_tokenizador, ngram_range = ngram)#,max_df = 0.95,min_df=0.01)
#VSM = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram, min_df=2, max_df=0.95, max_features=700)
#VSM.fit(text)
#VSM.vocabulary_


In [ ]:
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#ngram = (1,3) ## n-gramas de tamanho 1 (unigrama) até 3 (tri-grama)
#VSM = TfidfVectorizer(tokenizer=meu_tokenizador, ngram_range = ngram)#,max_df = 0.95,min_df=0.01)
#VSM = CountVectorizer(tokenizer=meu_tokenizador,ngram_range=ngram, min_df=2)#, max_df=0.95)#, max_features=500)
#VSM.fit(text)
#VSM.vocabulary_

remove ngrams irrelevantes

In [ ]:
def remove_ngrams_from_list(ngram_list):
  irrelevant_ngrams = []


  ngram_list = list(ngram_list)#inseri esse comando para transformar numpy array em lista, porque estava dando erro no trecho que remove os n-grams

  for k in range(len(irrelevant_ngrams)):
   if irrelevant_ngrams[k] in ngram_list:
      ngram_list.remove(irrelevant_ngrams[k])

  return ngram_list

Dividir o texto em conjunto de treinamento e de teste, com a proporção de 80% e 20%, respectivamente

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = BERT_preprocessing(text, tokenizer)
MAX_LEN = 128
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

k = 5
split = 0
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=7)
kfold = skf.split(input_ids, target)
split_index = []
for train, test in kfold:
  split_index.append([train, test])
X_train = [input_ids[i] for i in split_index[split][0]]
X_train_masks = [attention_masks[i] for i in split_index[split][0]]
y_train = [target[i] for i in split_index[split][0]]
X_test =  [input_ids[i] for i in split_index[split][1]]
X_test1 =  [text[i] for i in split_index[split][1]]
X_test_masks =  [attention_masks[i] for i in split_index[split][1]]
ID_test = [ID[i] for i in split_index[split][1]]
y_test = [target[i] for i in split_index[split][1]]
#X_train, X_test, y_train, y_test = train_test_split(text, target, train_size = 0.8, random_state=100)   ## 80% de treinamento. alterar train_size para o valor desejável. Exemplo: train_size=0.6 para 60% de treinamento
print(X_train[0])

[  101  2300 14173  1999  3019 20331  2104  2367  2968  2015  1024  2300
  1998  5800  6409  1998 25473  1010 18044  1998  2572 26387  1999  1996
 19550  3128 24813  1999  2670  4380  2024  2109  2005  7125  1998  8351
 15400  1012  1996  2005  4270  3749  1999  1996  3500  1011  2621  2161
  2003  3618  2084  1996  4111  9095  1010  2061  1996  2005  4270  2187
  2058  2003  2788  5296  1012 20787  5255  9754 10057 20435  1998  3727
  1996  5800  6436  1010 29170  2300 14173  1012  2023  2817  6461  2000
 24110 27351  2561  5800  1998  2300  6409  1998  1052  1010  1047  1998
 18699  1006  1018  1007  1006  1009  1007  8417  1999  1996 19550  2300
  2013  2019 23060 19565  2140  2007  3128 20787  1012  1999  1996 13441
  1010  1996  3128 24813  1010  2020  1045  1007]


In [ ]:
X_train = torch.tensor(X_train)

X_test = torch.tensor(X_test)
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
X_train_masks = torch.tensor(X_train_masks)
X_test_masks = torch.tensor(X_test_masks)
batch_size=8
      # Create an iterator of our data with torch DataLoader
train_data = TensorDataset(X_train, X_train_masks, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(X_test, X_test_masks, y_test)
validation_sampler = SequentialSampler(validation_data)
prediction_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## Vetoriza as sentenças

In [ ]:
def correlation(X1, lista, limiar):
  X1[X1>0] = 1
  dst = {}
  lst = {}
  for idx in range(X1.shape[1]-1):
   if lista[idx] not in lst.keys():
    for idx2 in range(idx+1, X1.shape[1]):
      if lista[idx2] not in lst.keys():
        minimo = np.linalg.norm(X1[:,idx] - X1[:,idx2])
        if minimo <= limiar:
          lst[lista[idx2]] = 0

  return lst


Este trecho é para alterar o vocabulário, ou seja, remove os n-gramas que são considerados irrelevantes para o problema.

## Imprimi a lista com os ngrams e sua frequencia

### Networkx e Plotly
* Construção de Redes k-NN
* Visualização Interativa de Grafos

In [ ]:
import plotly.graph_objects as go
import networkx as nx
from networkx.algorithms import community

### Sklearn
* Medidas de Similaridade

In [ ]:
from sklearn.neighbors import kneighbors_graph

In [ ]:
def show_graph_cluster(G):
  ### ARESTAS
  edge_x = []
  edge_y = []

  # adicionando as coordenadas
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  # definindo cor e estilo das arestas
  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=2, color='#888'),
      hoverinfo='none',
      mode='lines')

  ### VÉRTICES
  node_x = []
  node_y = []

  # adicionando as coordenadas
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)

  # definindo cor e estilo dos vértices
  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          size=10,
          line_width=2))


  # adicionando texto nos vértices
  node_text = []
  for node in G.nodes():
      node_text.append(G.nodes[node]['text'])
  node_trace.text = node_text

  # adicionando cores nos vértices de acordo com o cluster
  node_labels = []
  for node in G.nodes():
    node_labels.append(G.nodes[node]['cluster'])

  node_trace.marker.color = node_labels

  # visualizando!
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()


def show_graph_regularization(G):
  ### ARESTAS
  edge_x = []
  edge_y = []

  # adicionando as coordenadas
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  # definindo cor e estilo das arestas
  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=1, color='#888'),
      hoverinfo='none',
      mode='lines')

  ### VÉRTICES
  node_x = []
  node_y = []

  # adicionando as coordenadas
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)

  # definindo cor e estilo dos vértices
  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          showscale=True,
          colorscale='Reds',
          size=10,
          line_width=2))

  # adicionando texto nos vértices
  node_text = []
  for node in G.nodes():
      node_text.append(str(G.nodes[node]['f'])+'<br>'+G.nodes[node]['text'])
  node_trace.text = node_text

  # adicionando cores nos vértices de acordo com o label
  node_labels = []
  for node in G.nodes():
    node_labels.append(G.nodes[node]['f'][0])

  node_trace.marker.color = node_labels

  # visualizando!
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()


def simple_regularization(G,labels,max_iter=30):

  # inicializando
  for n in G.nodes():
    G.nodes[n]['f'] = np.array([0.0])
    if n in labels:
      G.nodes[n]['y'] = np.array([1.0])
      G.nodes[n]['f'] = np.array([1.0])

  for i in range(0,max_iter):

    # propagando
    diff = 0
    for node in G.nodes():
      if node in labels: continue
      f_new = np.array([0.0])
      count = 0
      for neighbor in G.neighbors(node):
        f_new += G.nodes[neighbor]['f']
        count += 1

      f_new /= count
      diff += np.linalg.norm(G.nodes[node]['f']-f_new)
      G.nodes[node]['f']=f_new

      if 'y' in G.nodes[node]:
        G.nodes[node]['f'] = G.nodes[node]['y']
    print("Iteration #"+str(i+1)+" Q(F)="+str(diff))


In [ ]:
df_document = pd.DataFrame(list(zip(ID, text, target)), columns=('UT (Unique WOS ID)', 'text', 'class'))
#df_document = pd.DataFrame(list(zip(ID, text, authors, X_gram, target)), columns=('UT (Unique WOS ID)', 'text', 'Authors', 'ngrams', 'class'))#tentando incluir authors na análise... mas deve precisar de algum pré-processamento antes
df_document

,UT (Unique WOS ID),text,class
0,WOS:000295854500036,water erosion in natural grassland under diffe...,1
1,WOS:000427469900010,soil organic carbon (soc) plays a critical rol...,1
2,WOS:000331652000025,soil erosion is one of the chief causes of agr...,1
3,WOS:000325848600005,the objective of this study was to quantify se...,1
4,WOS:000328262800007,in the watershed of the timboteua and buiuna s...,1
...,...,...,...
851,WOS:000382675700006,blumeria graminis (bgt) is a pathogenic fungus...,0
852,WOS:000425567400009,brazil is the first developing country to prov...,1
853,WOS:000279096000004,the availability of a large number of expresse...,0
854,WOS:000370492100019,because soil microbes drive many of the proces...,1


# Rede k-NN

### Gerando uma rede usando apenas documentos dos maiores clusters

In [ ]:
A = kneighbors_graph(input_ids, n_neighbors=5, metric="cosine")

In [ ]:
G = nx.Graph(A)

In [ ]:
cluster_id = 0
for clusters in community.label_propagation_communities(G):
  for doc_id in clusters:
    G.nodes[doc_id]['cluster'] = cluster_id
  cluster_id +=1

L_clusters = []
for index,row in df_document.iterrows():
  L_clusters.append(G.nodes[index]['cluster'])
df_document['cluster'] = L_clusters
df_document

,UT (Unique WOS ID),text,class,cluster
0,WOS:000295854500036,water erosion in natural grassland under diffe...,1,0
1,WOS:000427469900010,soil organic carbon (soc) plays a critical rol...,1,0
2,WOS:000331652000025,soil erosion is one of the chief causes of agr...,1,0
3,WOS:000325848600005,the objective of this study was to quantify se...,1,0
4,WOS:000328262800007,in the watershed of the timboteua and buiuna s...,1,0
...,...,...,...,...
851,WOS:000382675700006,blumeria graminis (bgt) is a pathogenic fungus...,0,0
852,WOS:000425567400009,brazil is the first developing country to prov...,1,0
853,WOS:000279096000004,the availability of a large number of expresse...,0,0
854,WOS:000370492100019,because soil microbes drive many of the proces...,1,0


### Treina um modelo Bert e imprime o resultado de classificação

In [ ]:
!pip install tqdm
from tqdm import trange

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
nb_labels = 2
from sklearn import linear_model, svm
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score, f1_score, precision_score
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=nb_labels)
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
           'weight_decay_rate': 0.01},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
           'weight_decay_rate': 0.0}
      ]

optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5,
                     )
                     #epsilon=1e-8,
                     #warmup=.1)


      # Store our loss and accuracy for plotting
train_loss_set = []


      # Number of training epochs
epochs = 5

for _ in trange(epochs, desc="Epoch"):

        ## TRAINING

        # Set our model to training mode
        model.train()
        # Tracking variables
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
          # Add batch to GPU
          batch = tuple(t.to(device) for t in batch)
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Clear out the gradients (by default they accumulate)
          optimizer.zero_grad()
          # Forward pass
          loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
          train_loss_set.append(loss.item())
          # Backward pass
          loss.backward()
          # Update parameters and take a step using the computed gradient
          optimizer.step()
          # Update tracking variables
          tr_loss += loss.item()
          nb_tr_examples += b_input_ids.size(0)
          nb_tr_steps += 1
        print("Train loss: {}".format(tr_loss/nb_tr_steps))




      # Put model in evaluation mode
model.eval()
      # Tracking variables
predictions , true_labels = [], []
      # Predict
for batch in prediction_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Store predictions and true labels
        predictions.append(logits)
        true_labels.append(label_ids)


from sklearn.metrics import matthews_corrcoef
matthews_set = []
for j in range(len(true_labels)):
        matthews = matthews_corrcoef(true_labels[j],
                 np.argmax(predictions[j], axis=1).flatten())
        matthews_set.append(matthews)

      # Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]
print("Accuracy score = ",accuracy_score(np.array(flat_true_labels), np.array(flat_predictions)))  ## acuracia  = (TP+TN)/(TP+TN+FP+FN)
print("Cohen Kappa = ",cohen_kappa_score(np.array(flat_true_labels), np.array(flat_predictions)))
print(f'Precision = {precision_score(np.array(flat_true_labels), np.array(flat_predictions))}')
print(f1_score(np.array(flat_true_labels), np.array(flat_predictions), average='macro'))
print(f1_score(np.array(flat_true_labels), np.array(flat_predictions), average='micro'))

print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))





Epoch:  20%|██        | 1/5 [00:21<01:24, 21.24s/it]

Train loss: 0.6947517713835073


Epoch:  40%|████      | 2/5 [00:39<00:58, 19.54s/it]

Train loss: 0.5993247538111931


Epoch:  60%|██████    | 3/5 [00:58<00:38, 19.11s/it]

...

Epoch:  80%|████████  | 4/5 [01:17<00:19, 19.04s/it]

Train loss: 0.4090825353944024


Epoch: 100%|██████████| 5/5 [01:36<00:00, 19.26s/it]

Train loss: 0.247567416060456


Accuracy score =  0.7441860465116279
Cohen Kappa =  0.4877487478001895
0.7433184099850767
0.7441860465116278
Classification accuracy using BERT Fine Tuning: 48.99%


In [ ]:
y_test = np.array(flat_true_labels)
y_pred = np.array(flat_predictions)
X_test1 = np.array(X_test1)
erros = []
for i in range(X_test1.shape[0]):
   if y_test[i]!=y_pred[i]:
     erros.append([ID_test[i], X_test1[i], y_pred[i], y_test[i]])
df_erros = pd.DataFrame(erros, columns=['ID', 'text', 'predicted', 'target'])
df_erros

,ID,text,predicted,target
0,WOS:000403676000010,aimsto address whether seed and seedling funct...,1,0
1,WOS:000352085500002,vetiver grass has been used to control erosion...,0,1
2,WOS:000401398600036,this manuscript analyzes the land use changes ...,1,0
3,WOS:000734064100001,the death syndrome of marandu grass (brachiari...,0,1
4,WOS:000081519000022,alho-macho (sisyrinchium platense johnst.) is ...,0,1
5,WOS:000254023100009,along the amazonian pioneer front in the brazi...,0,1
6,WOS:000418214300024,caatinga forests occupy an area of around one ...,0,1
7,WOS:000402662500022,the present study aimed to evaluate two rotati...,0,1
8,WOS:000248652800009,"araucaria angustifolia, also known as the para...",1,0
9,WOS:000264201600014,soil physical characteristics were evaluated o...,0,1
